In [1]:
import jax
from jax import numpy as jnp
import mujoco
from mujoco import mjx
import mediapy as media
from lib import calculate_fk, calculate_ik_step, calculate_ik

In [2]:
xml = """
<mujoco model="two_joint_arm">
    <compiler angle="degree" inertiafromgeom="true"/>
    <option timestep="0.01" gravity="0 0 -9.81"/>
    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom name="floor" pos="0 0 -0.1" size="1 1 0.1" type="plane" rgba="0.8 0.9 0.8 1"/>
        <body name="base" pos="0 0 0">
            <geom name="base" type="cylinder" size="0.05 0.02" rgba="0.2 0.2 0.2 1"/>
            <body name="upper_arm" pos="0 0 0.02">
                <joint name="shoulder" type="hinge" axis="0 0 1" range="-90 90"/>
                <geom name="upper_arm" type="capsule" fromto="0 0 0 0.2 0 0" size="0.02" rgba="0.7 0.7 0 1"/>
                <body name="lower_arm" pos="0.2 0 0">
                    <joint name="elbow" type="hinge" axis="0 1 0" range="-90 90"/>
                    <geom name="lower_arm" type="capsule" fromto="0 0 0 0.2 0 0" size="0.02" rgba="0 0.7 0.7 1"/>
                    <site name="end_effector" pos="0.2 0 0" size="0.01" rgba="1 0 0 1"/>
                </body>
            </body>
        </body>
        <body name="target" pos="0.3 0.1 0.2">
            <geom name="target" type="sphere" size="0.02" rgba="2 0 0 0.5"/>
        </body>
    </worldbody>
    <actuator>
        <motor joint="shoulder" ctrlrange="-1 1" ctrllimited="true"/>
        <motor joint="elbow" ctrlrange="-1 1" ctrllimited="true"/>
    </actuator>
    <sensor>
        <touch name="touch_sensor" site="end_effector"/>
    </sensor>
</mujoco>
"""

In [3]:
mj_model = mujoco.MjModel.from_xml_string(xml)
mj_data = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model)

mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)

print(mj_data.qpos, type(mj_data.qpos))
print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())

[0. 0.] <class 'numpy.ndarray'>
[0. 0.] <class 'jaxlib.xla_extension.ArrayImpl'> {CpuDevice(id=0)}


In [4]:
def is_in_range(x, lower, upper):
    return jnp.logical_and(x >= lower, x <= upper)

def touch_target(z_pos, target_z):
    if is_in_range(jnp.array(z_pos), 0.185, 0.188):
        return jnp.array([0, -1])
    else:
        return jnp.array([1, 0])

def cost(target_pos: jax.Array, current_pos: jax.Array) -> float:
    return jnp.power((target_pos - current_pos), 2)

In [5]:
def radians_to_normalized(angle, angle_min, angle_max):
    return 2 * (angle - angle_min) / (angle_max - angle_min) - 1
  

In [9]:
import numpy as np

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True


mujoco.mj_resetData(mj_model, mj_data)
min_angle = -jnp.pi / 2
max_angle = jnp.pi / 2

duration = 10 # (seconds)
framerate = 60  # (Hz)
r = 0.2
desired_position = jnp.array([0.3, 0.1, 0.2])
initial_q = np.random.uniform(-1, 1, size=(2,))
ik = calculate_ik(initial_q=jnp.array(initial_q), length=r, desired_position=desired_position, alpha=1e-4)
normalized_ik = jnp.array([radians_to_normalized(ik.at[0].get(), min_angle, max_angle), radians_to_normalized(ik.at[1].get(), min_angle, max_angle),]) 
print(ik)
print(f'Desired joint config: {normalized_ik}')
print(f'Initial guess: {initial_q}')
frames = []

mj_data.ctrl = normalized_ik
print(normalized_ik)
while mj_data.time < duration:
  mujoco.mj_step(mj_model, mj_data)
  
  if len(frames) < mj_data.time * framerate:
    renderer.update_scene(mj_data, scene_option=scene_option)
    pixels = renderer.render()
    frames.append(pixels)
print("Sensor data")
# Simulate and display video.
media.show_video(frames, fps=framerate)

0.32604325
0.3257743
0.32550547
0.3252367
0.32496795
0.3246993
0.32443073
0.32416221
0.32389382
0.32362548
0.3233572
0.323089
0.3228209
0.3225529
0.32228497
0.32201713
0.32174933
0.32148165
0.321214
0.32094646
0.320679
0.32041162
0.32014433
0.31987712
0.31961
0.31934297
0.31907603
0.3188092
0.31854242
0.31827578
0.3180092
0.31774265
0.31747627
0.31720993
0.31694373
0.31667757
0.31641153
0.31614557
0.31587973
0.31561399
0.31534833
0.31508273
0.31481728
0.31455192
0.31428662
0.3140215
0.3137564
0.3134914
0.31322652
0.3129617
0.31269705
0.31243247
0.312168
0.31190366
0.31163937
0.3113752
0.31111112
0.31084713
0.31058326
0.3103195
0.31005588
0.30979234
0.30952892
0.30926558
0.30900237
0.30873924
0.30847624
0.30821335
0.30795056
0.3076879
0.30742535
0.3071629
0.3069006
0.3066384
0.30637622
0.30611426
0.3058524
0.30559066
0.30532905
0.30506748
0.30480608
0.3045448
0.30428365
0.30402258
0.30376163
0.30350083
0.30324012
0.30297956
0.30271912
0.3024588
0.3021986
0.3019385
0.30167854
0.3014187
0

This browser does not support the video tag.


In [8]:
arr = jnp.array([[1,2,3], [1, 23, 4]])
arr

Array([[ 1,  2,  3],
       [ 1, 23,  4]], dtype=int32)

In [12]:
arr.at[0].get().sum()

Array(6, dtype=int32)